In [3]:
import random
import json

In [4]:
from openai import OpenAI

# filepath: /path/to/your/file.py
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [5]:
#read_qa_pairs
qa_path = "../data/qa_list.json"
with open(qa_path, 'rb') as f:
    qa_list = json.load(f)
qa_list[:2]

['Dilek tutmana yardımcı olacak meyve nedir?: WishNe',
 'Asla gelmediği için izleyemediğimiz Coen Biraderler filmi nedir?: Yurtiçi Kargo']

In [6]:
data_path = "../data/dad_joke_samples.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]
dataset[0]

{'messages': [{'role': 'system',
   'content': 'You are a dad joke expert. Create dad jokes in accordance with the user input.'},
  {'role': 'user', 'content': 'Bana bir espri yapar mısın?'},
  {'role': 'assistant',
   'content': 'Dilek tutmana yardımcı olacak meyve nedir?'},
  {'role': 'user', 'content': 'Nedir?'},
  {'role': 'assistant', 'content': 'WishNe.'}]}

In [7]:
def find_dict_index(lst, search_string):
    for index, dictionary in enumerate(lst):
        if any(search_string in str(value) for value in dictionary.values()):
            return index
    return -1 

find_dict_index(dataset, "Beyazıt")

29

In [8]:
user_inputs = [sample['messages'][1]['content'] for sample in dataset]
user_inputs[:2]

['Bana bir espri yapar mısın?',
 'Yurtiçi Kargo ile ilgili bir espri yapar mısın?']

In [9]:
examples = [f"user_input:\n{user_input}\nqa_pair:\n{qa_pair}" for user_input, qa_pair in zip(user_inputs, qa_list)]
print(examples[29])

user_input:
R harflerini söyleyemeyen Beyazıt Öztürk ile ilgili bir espri yapar mısın?
qa_pair:
Kurtlar Vadisi’nde neden hiç yük taşıma sahnesi yoktur?: Çünkü iki kişinin bindiği tır, tır değildir.


In [10]:
system_prompt = """
Given a user_input and qa_pair of a dad joke written in Turkish, generate a thinking steps in plain text with a few sentences from your point of view  as if you are right before to create this joke. 
Here a few examples:
user_input:
Bana bir espri yapar mısın?
qa_pair:
Dilek tutmana yardımcı olacak meyve nedir?: WishNe
reasoning:
Kulanıcı benden bir espri yapmamı istiyor, özellikle bir tema belirlememiş o yüzden istediğim konuda bir espri yapabilirim. Türkçe ve İngilizce dillerini kullanarak bir kelime şakası yapabilirim. İngilizcedeki cherry meyvesi Türkçe'de vişne olarak yazılır. İlk hecesinin okunuşu ingilizcedeki wish kelimesinin okunuşu gibidir. Buradan yola çıkarak bir kelime şakası yapabilirim. Dilek tutmak ve vişne kelimesi arasında bir bağlantı kurarak bir espri yapabilirim. Türkçedeki vişne kelimesinin ingilizcedeki Wish ile fonetik bağlantısını göstermek için vişnenin ilk hecesini ingilizce olarak yazabilirim. Cevabı WishNe olan bir soru oluşturabilirim. Soruyu oluşturuken de vişne ve wish kelimelerinin benzerliğine dikkat çekmem gerekir. Wish kelimesinin Türkçe anlamı dilek tutmaktır, vişne bir meyve ismidir. Bu yüzden dilek tutmana yardımcı olacak meyve nedir? sorusunu sorabilirim. 

user_input:
Bana bir espri yapar mısın?
qa_pair:
Bir halı saha maçında antrenör Beyazıt Öztürk’e ‘bu gölü kim atacak?’ diye sorduğunda ne cevap vermiştir?: Ali Atay
reasoning:
Kullanıcı benden bir espri yapmamı istiyor. Esprinin konu ve temasını belirlemeden serbest bir şekilde yaratabileceğim için aklıma birkaç farklı yaklaşım geliyor. Halı saha maçları ve futbol, Türkiye'deki popüler konulardan biri. Ayrıca, Türk komedyen ve televizyoncu Beyazıt Öztürk'ü esprime dahil ederek tanıdık bir figür yaratabilirim. "Ali Atay" ismi, hem bir aktör hem de futbol terimi olan "gol atma" fikriyle bağlantılı. Beyazıt Öztürk'ün r harflerini söylemekte zorlandığı biliniyor. 'r' harflerini söyleyemeyen insanlar genelde bu harfi 'y' gibi söylerler. Bu yüzden Ali Atay ismindeki 'y' harfini 'r' harfine çevirerek bir kelime oyunu yapabilirim. Böylelikle Ali Atay ismi Ali Atar oluyor, bu da türkçedeki atmak fiiliyle benzer bir ses yapısı oluşturuyor. Bu yüzden antrenör Beyazıt Öztürk'e 'bu gölü kim atacak?' diye sorduğunda ne cevap vermiştir? sorusunu sorabilirim. Cevap olarak da Ali Atay diyebilirim. Beyazıt Öztürk bu soruya cevap vermek istediğinde Ali Atar diyemeyeceği için Ali Atay diyebilir ve Ali Atay ünlü bir oyuncu olduğu için bu komik bir durum oluşturur. 
"""

In [11]:
def joke_resoning_generator(user_input,system_prompt):
    user_content = f"""
    {user_input}
    reasoning:
    """
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": user_content
            }
        ]
    ) 

    return completion.choices[0].message.content

In [12]:
print(joke_resoning_generator(examples[9],system_prompt))

Kullanıcı benden kapalı mekanlarla ilgili bir espri yapmamı istiyor. İsteği üzerine düşünürken, kapalı mekanların genellikle özel ve kişisel anların yaşandığı yerler olduğunu fark ediyorum. Buradan yola çıkarak, insanların özel hayatlarıyla ilgili komik bir durumdan esinlenebilirim. Kapalı mekan denilince akla gelen alanlardan biri, insanların en mahrem anlarını ve kişisel ilişkilerini yaşadıkları yatak odaları olabilir. Genellikle bu özel anlar haricinde açıkça konuşulmasa da, toplum içinde cinsel hayat kavramı her zaman espri konusu olmuştur. Ayrıca, "rol oynamak" ve "gösterim" kelimeleri tiyatral bir atmosferi çağrıştırıyor, ki bu da kapalı alan temasıyla uyumlu. Herkesin bir rolü olduğu, yani cinsel ilişkide rollerin ve beklentilerin yer aldığı duruma bir gönderme yaparak, kişisel hayatların genelde eğlenceli bir biçimde başkaları için komik malzeme oluşturduğunu ifade edebilirim. Buradan, kapalı mekanlar ve herkesin rol oynadığı komik durumları birleştirerek, "Cinsel Hayatım" ifad

In [13]:
print(qa_list[9])

Kapalı mekanlardan geçen herkesin bir rolü oynadığı sonu hep başkalarının kahkahası ile biten çok kısa gösterimlere ne denir?: Cinsel Hayatım


In [14]:
from tqdm import tqdm

# Initialize reasonings list
reasonings = []

# Process examples with progress bar
try:
    for example in tqdm(examples, desc="Generating reasonings"):
        reasonings.append(joke_resoning_generator(example, system_prompt))
except KeyboardInterrupt:
    print("Process interrupted. Progress saved in memory.")

# Display the first two reasonings
print(reasonings[:2])

Generating reasonings: 100%|██████████| 60/60 [07:04<00:00,  7.08s/it]

['Kullanıcı benden bir espri yapmamı istiyor, belirli bir konuya odaklanmamış olduğu için geniş bir yaratıcılık alanım var. Kelime oyunları genellikle hem basit hem de etkili olabilir. İngilizce ve Türkçe arasında bir benzerlik bulmak eğlenceli olabilir. İngilizcede "wish" kelimesi, Türkçede "dilek" anlamına gelir. Aynı zamanda Türkçe’de "vişne" kelimesinin ilk hecesi "wish" gibi telaffuz edilir. Buradan hareketle, bir kelime oyunu oluşturabilirim: "dilek tutmaya yardımcı olan bir meyve" şeklinde bir soru sorarak "wish" kelimesinin Türkçe fonetik benzerliğinden yararlanırım. Sorunun cevabı ise fonetik benzerliği belirginleştirmek adına "WishNe" olur. Bu espriyi yaparken, iki dilin ses benzerliklerini kullanarak bir gülümseme yaratabileceğimi düşünüyorum. Ayrıca meyve isimlerini esprinin merkezine koymak, günlük hayatta karşılaşabileceğimiz objelerle ilgili bir mizah oluşturuyor.', 'Kullanıcı benden Yurtiçi Kargo hakkında bir espri yapmamı istiyor, bu da bana espriyi kargo teslimatlarıy

In [15]:
#save reasonings as a pickle file
output_path = "../data/reasonings.json"
with open(output_path, "w") as f:
    json.dump(reasonings, f)